In [ ]:
import re
import time
import datetime
import paho.mqtt.client as mqtt
from typing import NamedTuple
from influxdb import InfluxDBClient
from influxdb.client import InfluxDBClientError


#Set Up MQTT Broker 
mqtt_broker = "test.mosquitto.org"
topic = "SmartHomeGrp1/+/+"
regex = "SmartHomeGrp1/([^/]+)/([^/]+)"
my_mqtt = None

#InfluxDB Configuration 
USER = 'root'
PASSWORD = 'root'
DBNAME = 'DB1'
HOST = 'localhost'
PORT = 8086
dbclient = None

#Setting variable types 
class SensorData(NamedTuple):
        location: str
        measurement: str
        value: float

#Connecting and Subscribng to MQTT Topics Temp, Humidity, Pressure and Fan
def on_connect(client, userdata, flags, rc):
    print('Connected with result code ' + str(rc))
    client.subscribe(topic)

#Convert sensor_data.value from String to float
def parse_mqtt(topic, payload):
    match = re.match(regex, topic)
    if match:
        location = match.group(1)
        measurement = match.group(2)
        if measurement == 'status':
            return None
        return SensorData(location, measurement, float(payload))
    else:
        return None

#Write json for Influxdb
def send_data_influxdb(data):
    json_body = [
    {
            'time': time.strftime("%Y-%-m-%d %H:%M:%S", time.gmtime()),
            'measurement': data.measurement,
            'tags': {
                'location': data.location,
            },
            'fields': {
                'value': data.value
            }
        }
    ]
    dbclient = InfluxDBClient(HOST, 8086, USER, PASSWORD, DBNAME)
    print (json_body)
    dbclient.write_points(json_body)

#Receive Temperature, Humidity and Pressure data
def onMessage(client, userdata, message):
    print("%s %s" %(message.topic, message.payload))
    data = parse_mqtt(message.topic, message.payload.decode('utf-8'))
    if data is not None:
        send_data_influxdb(data)

def main():
    my_mqtt = mqtt.Client()
    my_mqtt.on_connect = on_connect
    my_mqtt.on_message = onMessage
    my_mqtt.connect(mqtt_broker, port = 1883)
    my_mqtt.loop_forever()

if __name__ == '__main__':
        main()
